# Toronto Bike Share Analysis
This notebook is part of a broader analysis of Toronto Bike Share usage, and what it can tell about the impact to traffic if certain bicycle infrastructure is removed in Toronto.
The final story can be found here:
This is the calculation of some descriptive statistics used in the story. The story also includes some analysis performed in QGIS. 


In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import psycopg
import plotly.express as px

# Load Environment Variables
load_dotenv()
if os.environ['ENV_FLG']: print('Environment variables loaded properly.')
else: print('ERROR: Environment variables failed to load.')

Environment variables loaded properly.


## Import Data

In [2]:
def query_db(query):
    # Execute query on PostgreSQL database and return a pandas dataframe
    db_name = os.environ['DB_NAME']
    db_user = os.environ['DB_USER']
    db_pass = os.environ['DB_PASS']
    db_host = os.environ['DB_HOST']
    db_port = os.environ['DB_PORT']

    engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')
    data_frame = pd.read_sql_query(query, engine)
    return data_frame

data = query_db('select * from bike_trips;')

In [3]:
# Filter columns
columns = ['trip_id','trip_duration','start_station_id','start_time','end_station_id','end_time','user_type']
X = data[columns]
X.head()

,trip_id,trip_duration,start_station_id,start_time,end_station_id,end_time,user_type
0,30841620,657,7084,2024-08-03 16:58:00,7174.0,2024-08-03 17:09:00,Casual Member
1,30841621,4348,7354,2024-08-03 16:58:00,7354.0,2024-08-03 18:11:00,Casual Member
2,30841622,976,7036,2024-08-03 16:58:00,7098.0,2024-08-03 17:15:00,Casual Member
3,30841624,1660,7788,2024-08-03 16:59:00,7171.0,2024-08-03 17:26:00,Casual Member
4,30841626,641,7161,2024-08-03 16:59:00,7791.0,2024-08-03 17:09:00,Casual Member


In [4]:
# Check missing values
missing_val_count_by_column = (X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

end_station_id    4318
dtype: int64


## Descriptive Analysis
### Statistics
- Total trips (2022-2024)
- Average trips per day 
### Plots
1. Trips per month over 2022-2024 (to see growth in usage)
2. Average trips per month (to compare winter usage)
3. Average trips by day of week (to determine commuter usage)
4. Trips by hour of day (to determine commuter usage)

In [5]:
# Overall: Total Trips (2022-2024)
print(f'Total Trips (01-2022 to 09-2024):\t{len(X):,}')

# Overall: Avg Trips per Day 
trips_per_day = X.groupby(X.start_time.dt.date)['trip_id'].count()
mean_trips_per_day = trips_per_day.mean()
print(f'Avg trips per day:\t\t\t{mean_trips_per_day:,.0f}')

# Trips per Year
print(f'Total trips 2024:\t\t{len(X[X.start_time.dt.year == 2024]):,}')
print(f'Total trips 2023:\t\t{len(X[X.start_time.dt.year == 2023]):,}')
print(f'Total trips 2022:\t\t{len(X[X.start_time.dt.year == 2022]):,}')

# Trips per Day
print(f'Avg trips per day (2024):\t\t{(X[X.start_time.dt.year == 2024].groupby(X.start_time.dt.date)['trip_id'].count()).mean():,.0f}')
print(f'Avg trips per day (2023):\t\t{(X[X.start_time.dt.year == 2023].groupby(X.start_time.dt.date)['trip_id'].count()).mean():,.0f}')
print(f'Avg trips per day (2022):\t\t{(X[X.start_time.dt.year == 2022].groupby(X.start_time.dt.date)['trip_id'].count()).mean():,.0f}')

Total Trips (01-2022 to 09-2024):	15,675,998
Avg trips per day:			15,614
Total trips 2024:		5,342,388
Total trips 2023:		5,713,141
Total trips 2022:		4,620,469
Avg trips per day (2024):		19,498
Avg trips per day (2023):		15,652
Avg trips per day (2022):		12,659


In [6]:
# Trips during 'rush hours' on Weekdays
mask_morn_rush_hour = ((X.start_time.dt.time >= pd.Timestamp('07:00:00').time()) & (X.start_time.dt.time <= pd.Timestamp('09:00:00').time()))   # Define morning rush hour times
mask_aftn_rush_hour = ((X.start_time.dt.time >= pd.Timestamp('16:00:00').time()) & (X.start_time.dt.time <= pd.Timestamp('18:00:00').time()))   # Define afternoon rush hour times
X_rush_hour_trips = X.loc[mask_morn_rush_hour | mask_aftn_rush_hour]

rush_hour_trips_per_day = X_rush_hour_trips.groupby(X.start_time.dt.date)['trip_id'].count()
mean_rush_hour_trips_per_day = rush_hour_trips_per_day.mean()
print(f'Total trips during \'rush hour\': \t{rush_hour_trips_per_day.sum():,}')
print(f'% of all trips during \'rush hour\': \t{rush_hour_trips_per_day.sum()/len(X):,.2f}')
print(f'Average trips during \'rush hour\' per day: \t{mean_rush_hour_trips_per_day:,.0f}')

Total trips during 'rush hour': 	4,326,182
% of all trips during 'rush hour': 	0.28
Average trips during 'rush hour' per day: 	4,309


In [ ]:
# Plot: Trips by Month, 2022-2024
plt_trips_histogram = px.histogram(X, x="start_time", y="trip_id", histfunc="count",
                                   template="simple_white", title="Bike Share Ridership, 01-2022 through 09-2024",
                                   labels={
                                       "start_time": "Month / Year",
                                       "trip_id": "Total trips"
                                   })
plt_trips_histogram.update_traces(xbins_size="M1")
plt_trips_histogram.update_xaxes(dtick="M1")
plt_trips_histogram.show()

In [ ]:
# Plot: Average Trips per Month
trips_by_year_month = X.groupby([X.start_time.dt.year, X.start_time.dt.month])['trip_id'].count().rename_axis(["year","month"])
mean_trips_month = trips_by_year_month.reset_index().groupby('month')['trip_id'].mean()

plt_trips_by_month = px.bar(mean_trips_month)
plt_trips_by_month.show()

In [ ]:
# Plot: Trips by Day of the Week
trips_by_day = X.groupby(X.start_time.dt.day_of_week)['trip_id'].count()
plt_trips_by_day = px.bar(trips_by_day)
plt_trips_by_day.show()

In [ ]:
# Trip Starts by Hour of Day
trips_by_hour = X.groupby(X.start_time.dt.hour)['trip_id'].count()
plt_trips_by_hour = px.bar(trips_by_hour, template="simple_white",
                           labels={
                               "start_time": "Trip Start Time"
                           })
plt_trips_by_hour.update_xaxes(tickformat="%H:%M", dtick="H1")
plt_trips_by_hour.show()

## Statistics from Buffer Analysis in QGIS

### Stations within 400m buffer of affected bike lanes
- 211 stations out of 852
- **24.77% of stations**

### Bike capacity of stations within 400m buffer of affected bike lanes
- 4419 bikes out of 16478
- **26.82% of rental bikes**

In [7]:
stations_in_400m_buffer = query_db('select station_id FROM stations_in_400m;')

trips_affected_stations = X[
    (X.start_station_id.isin(stations_in_400m_buffer.station_id)) |
    (X.end_station_id.isin(stations_in_400m_buffer.station_id))
]

ratio_trips_affected = len(trips_affected_stations)/len(X)
print(f'Percentage of trips to/from affected stations: \t{ratio_trips_affected*100:.2f}%')
print(f'Average trips per day affected: \t\t{mean_trips_per_day * ratio_trips_affected:,.0f}')



Percentage of trips to/from affected stations: 	57.63%
Average trips per day affected: 		8,999


In [8]:
# Trips during rush hour - on weekdays
X_rush_hour_weekdays = X_rush_hour_trips[X_rush_hour_trips.start_time.dt.day_of_week <=4]

# Trips during rush hour - on weekdays - to/from stations within 400m of Bloor, Yonge, University
X_rush_hour_weekdays_affected = X_rush_hour_weekdays[
    (X_rush_hour_weekdays.start_station_id.isin(stations_in_400m_buffer.station_id)) |
    (X_rush_hour_weekdays.end_station_id.isin(stations_in_400m_buffer.station_id)) 
]

print(len(X_rush_hour_trips), len(X_rush_hour_weekdays), len(X_rush_hour_weekdays_affected))
print(len(X_rush_hour_weekdays_affected)/len(X)*100)

X_rush_hour_weekdays_affected_per_day = X_rush_hour_weekdays_affected.groupby(X_rush_hour_weekdays_affected.start_time.dt.date)['trip_id'].count()
X_rush_hour_weekdays_affected_per_day = X_rush_hour_weekdays_affected_per_day.mean()
print(f'Average trips per day during the week during rush hour starting at or ending at stations near the affected lanes: {X_rush_hour_weekdays_affected_per_day:,.0f}')

4326182 3457830 2203826
14.0586009260782
Average trips per day during the week during rush hour starting at or ending at stations near the affected lanes: 3,078
